<a href="https://colab.research.google.com/github/domukas44/gmm/blob/main/GMM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openimages

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 74.5 MB/s eta 0:00:00


In [ ]:
from openimages.download import download_dataset

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import matplotlib.pyplot as plt
import os

In [ ]:
data_dir = "data"

number_for_samples = 334
classes = ["Apple", "Motorcycle", "Snowboard"]

In [ ]:
!nvidia-smi

Thu Apr 20 17:17:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
# !rm -rf data

In [ ]:
print("Downloading is starting...")
download_dataset(data_dir, classes, limit=number_for_samples, annotation_format="pascal")

100%|██████████| 22/22 [00:00<00:00, 1138.18it/s]


{'apple': {'images_dir': 'data/apple/images',
  'annotations_dir': 'data/apple/pascal'},
 'motorcycle': {'images_dir': 'data/motorcycle/images',
  'annotations_dir': 'data/motorcycle/pascal'},
 'snowboard': {'images_dir': 'data/snowboard/images',
  'annotations_dir': 'data/snowboard/pascal'}}

In [ ]:
import torch
from torch.autograd import Variable
from torchvision import models

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
writer = SummaryWriter() 

number_of_classes = 2

#model:
retinaNet_model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True) #, num_classes = 3)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:00<00:00, 196MB/s]


In [ ]:
import glob2
import xml.etree.ElementTree as ET
import numpy as np
import random

meta_data_files = glob2.glob("{}/*/pascal/*xml".format(data_dir))
random.shuffle(meta_data_files)
meta_data_files

['data/apple/pascal/832449fdd1709a3a.xml',
 'data/apple/pascal/3e810338e95a9cee.xml',
 'data/apple/pascal/dd95ee5a42971197.xml',
 'data/snowboard/pascal/737da6760dee8c6e.xml',
 'data/apple/pascal/246fdaba724b926f.xml',
 'data/motorcycle/pascal/01790e9094c0bd92.xml',
 'data/motorcycle/pascal/02b190ed1616a58c.xml',
 'data/apple/pascal/004e8f8aa9860565.xml',
 'data/apple/pascal/b3cbc32556e1e877.xml',
 'data/apple/pascal/35666ba431eab8d7.xml',
 'data/motorcycle/pascal/023b916a1dfe4c43.xml',
 'data/motorcycle/pascal/019123b87fc4d34c.xml',
 'data/apple/pascal/0fb342b1e5f088c2.xml',
 'data/apple/pascal/75533c5e58f7fca8.xml',
 'data/apple/pascal/5de3ff07c845622e.xml',
 'data/snowboard/pascal/16b2aa7e3e7e4757.xml',
 'data/apple/pascal/8f2c0ef78304476b.xml',
 'data/apple/pascal/16321dae4a13d726.xml',
 'data/motorcycle/pascal/022c8644abab2706.xml',
 'data/apple/pascal/3c420f8ab88a11d9.xml',
 'data/apple/pascal/0deb94399a94468a.xml',
 'data/motorcycle/pascal/01868f7ebe680e2f.xml',
 'data/apple/pas

In [ ]:
files = []
locations = []
labels = []
 
for meta_file in meta_data_files:
    tree = ET.parse(meta_file)
    root = tree.getroot()
 
    file_path = root.find('path').text
    width = int(root.find('size').find('width').text)
    height = int(root.find('size').find('height').text)
    
    for obj in root.findall('object'):
        xmin = int(obj.find('bndbox').find('xmin').text)
        xmax = int(obj.find('bndbox').find('xmax').text)
        ymin = int(obj.find('bndbox').find('ymin').text)
        ymax = int(obj.find('bndbox').find('ymax').text)
        if xmax - xmin > 0 and ymax - ymin > 0:
            bounding_box = [xmin, ymin, xmax, ymax, width, height]
            locations.append(bounding_box)
            files.append(file_path)
            label = obj.find('name').text
            if label == "apple":
              labels.append(1)
            elif label == "motorcycle":
              labels.append(2)
            elif label == "Snowboard":
              labels.append(3)

In [ ]:
locations = np.array(locations)
labels = torch.tensor(labels)

In [ ]:
print(len(locations), len(files))
print(len(labels), len(files))

842 842
798 842


In [ ]:
Y = locations[:, :4].astype(np.int) ## Pasiimame koordinates 
Y

array([[ 650,  569,  885,  816],
       [ 401,  548,  640,  795],
       [  14,   52, 1010,  990],
       ...,
       [ 108,   99,  510,  516],
       [ 359,  556,  481,  656],
       [ 428,  711,  569,  839]])

In [ ]:
import torch
from torch.autograd import Variable
from torchvision import models
from torchvision.transforms import transforms
from  skimage import io

In [ ]:
#scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [ ]:
def read_img(file_name):
    img = Image.open(file_name)
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    return t_img.float()

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
import math

class AllDataset(object):
    def __init__(self, files, y, labels_, transforms = None):
        self.root = root
        self.transforms = transforms
        self.files = files
        self.bounding_boxes = y
        self.labels_ = labels_

    def __getitem__(self, idx):
        # load images ad masks
        img_path = self.files[idx]
        
        img = io.imread(img_path)
        w, h, _ = img.shape
        img.shape

        obj_ids = 1
        
        num_objs = 1
        
        xmin, ymin, xmax, ymax = self.bounding_boxes[idx,:4]
        boxes = []
        boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.tensor(boxes, dtype=torch.float32)

        labels = torch.tensor([self.labels_[idx]])

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        mask = np.zeros((h, w))
        mask[xmin:xmax, ymin:ymax] = 1
        masks = mask == 1

        masks = torch.as_tensor(masks, dtype=torch.uint8)
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        #target["masks"] = masks
        #target["area"] = area
        #target["iscrowd"] = iscrowd
        
        target = target
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.files)

In [ ]:
dataset = AllDataset(files,  Y, labels)
#dataset[0][0].shape

dataset[0]

(array([[[144, 147, 162],
         [138, 141, 156],
         [133, 136, 151],
         ...,
         [115, 115, 113],
         [115, 115, 113],
         [114, 114, 112]],
 
        [[141, 144, 159],
         [138, 141, 156],
         [136, 139, 154],
         ...,
         [117, 117, 115],
         [116, 116, 114],
         [115, 115, 113]],
 
        [[139, 142, 157],
         [138, 141, 156],
         [139, 142, 157],
         ...,
         [116, 116, 114],
         [115, 115, 113],
         [114, 114, 112]],
 
        ...,
 
        [[133,  97,  49],
         [131,  94,  49],
         [127,  90,  45],
         ...,
         [126,  90,  54],
         [122,  86,  50],
         [119,  83,  47]],
 
        [[130,  93,  48],
         [128,  91,  46],
         [126,  89,  44],
         ...,
         [123,  87,  51],
         [121,  85,  49],
         [120,  84,  48]],
 
        [[139, 102,  58],
         [137, 100,  56],
         [136,  99,  55],
         ...,
         [123,  87,  51],
  

In [ ]:
dataset[0][0].size, len(dataset)

(2678784, 842)

In [ ]:
# pick one image from the test set
img, _ = dataset[0]
# put the model in evaluation mode


#retinaNet_model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
#retinaNet_model.cuda()
retinaNet_model.eval()

with torch.no_grad():
    prediction = retinaNet_model([img.to(device)])

AttributeError: ignored

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
prediction

In [ ]:
def to_coord(pred, shape):
    _, w, h = shape
    x0 = max(int(pred[0]), 0)
    x1 = min(int(pred[1]), w)
    y0 = max(int(pred[2]), 0)
    y1 = min(int(pred[3]), h)   
    
    return [x0, y0, x1 - x0, y1 - y0]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
fig, ax = plt.subplots()

ax.imshow(img.mul(255).permute(1, 2, 0).byte().numpy())

for pred in prediction[0]['boxes']:
    pred = pred.cpu().numpy()
    predicted = to_coord(pred, img.shape)
    rect = patches.Rectangle((predicted[0], predicted[1]), predicted[2], predicted[3], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)



plt.show()